# Step 01: Export to RDM

This notebook exports all analysis results and groups to a Results Data Mart (RDM).

**Tasks:**
- Retrieve Export to RDM batch from Stage_01/Step_03
- Review export job configuration
- Submit export job to Moody's API
- Track job completion status

**Prerequisites:**
- All Analysis jobs must be complete
- All Grouping and Grouping Rollup jobs must be complete
- Export RDM Name must be configured in Metadata

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.batch import submit_batch, get_batch_jobs, read_batch
from helpers.database import execute_query
from helpers.irp_integration import IRPClient
from helpers.constants import BatchType

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_01_Export_to_RDM.ipynb')

# Display context
ux.header("Export to RDM")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"Step tracking initialized for '{context.step_name}'")

## 2) Retrieve Export to RDM Batch

In [ ]:
# Retrieve Export to RDM batch from Stage_01/Step_03
ux.subheader("Retrieve Export to RDM Batch")

# Query for Stage_01/Step_03 step run
query = """
    SELECT sr.id, sr.step_id, sr.run_num, sr.output_data, sr.completed_ts
    FROM irp_step_run sr
    INNER JOIN irp_step s ON sr.step_id = s.id
    INNER JOIN irp_stage sg ON s.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
      AND sg.stage_num = 1
      AND s.step_num = 3
      AND sr.status = 'COMPLETED'
    ORDER BY sr.completed_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    raise ValueError("Batch creation step not found - please complete Stage_01/Step_03 first")

output_data = result.iloc[0]['output_data']
batches = output_data.get('batches', {})

if BatchType.EXPORT_TO_RDM not in batches:
    raise ValueError(f"Export to RDM batch not found. Available: {list(batches.keys())}")

export_batch_id = int(batches[BatchType.EXPORT_TO_RDM])

ux.success(f"Retrieved Export to RDM batch: ID={export_batch_id}")
step.log(f"Retrieved Export to RDM batch: ID={export_batch_id}")

## 3) Review Export Configuration

In [ ]:
# Verify batch status and display job information
ux.subheader("Verify Batch Status")

# Read batch details
batch = read_batch(export_batch_id)

batch_info = [
    ["Batch ID", batch['id']],
    ["Batch Type", batch['batch_type']],
    ["Status", batch['status']],
    ["Created", batch['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(batch_info, headers=["Property", "Value"])

# Get jobs in batch (should be 1 job)
jobs = get_batch_jobs(export_batch_id)
job_count = len(jobs)

ux.info(f"\nTotal jobs: {job_count}")

# Show export configuration
if job_count > 0:
    ux.info("\nExport Configuration:")
    
    for job in jobs:
        config_query = "SELECT job_configuration_data FROM irp_job_configuration WHERE id = %s"
        config_result = execute_query(config_query, (job['job_configuration_id'],))
        
        if not config_result.empty:
            config = config_result.iloc[0]['job_configuration_data']
            
            rdm_name = config.get('rdm_name', 'N/A')
            server_name = config.get('server_name', 'N/A')
            analysis_names = config.get('analysis_names', [])
            analysis_count = config.get('analysis_count', 0)
            group_count = config.get('group_count', 0)
            
            export_info = [
                ["Target RDM", rdm_name],
                ["Server", server_name],
                ["Analyses to Export", analysis_count],
                ["Groups to Export", group_count],
                ["Total Items", len(analysis_names)]
            ]
            ux.table(export_info, headers=["Property", "Value"])
            
            # Show preview of items to export
            ux.info("\nItems to export (first 15):")
            for name in analysis_names[:15]:
                ux.info(f"  - {name}")
            if len(analysis_names) > 15:
                ux.info(f"  ... and {len(analysis_names) - 15} more")

step.log(f"Verified batch: {job_count} job(s) ready for submission")

## 4) Submit Export to RDM Batch

In [ ]:
# Submit batch to Moody's API
ux.subheader("Submit Batch to Moody's")

ux.info("")
ux.info("Submission Process:")
ux.info("  - Analysis and group names will be resolved to URIs")
ux.info("  - Results will be exported to the configured RDM")
ux.info(f"  - Target RDM: {rdm_name}")
ux.info(f"  - Server: {server_name}")
ux.info("  - Job will transition to SUBMITTED status")
ux.info("  - Batch will transition to ACTIVE status")
ux.info("")

# Submit
ux.info("\nSubmitting batch...")

# Pass step.step_id to associate batch with this step (not the creation step)
submit_result = submit_batch(export_batch_id, IRPClient(), step_id=step.step_id)

# Display results
ux.success(f"\nBatch submission completed")
ux.info(f"  Submitted: {submit_result['submitted_jobs']} job(s)")
ux.info(f"  Status: {submit_result['batch_status']}")

# Check for errors
failed_count = len([j for j in submit_result['jobs'] if 'error' in j])
if failed_count > 0:
    ux.warning(f"\n{failed_count} job(s) failed to submit")
    for job_result in submit_result['jobs']:
        if 'error' in job_result:
            ux.error(f"  Job {job_result['job_id']}: {job_result['error']}")

step.log(f"Batch submitted: {submit_result['submitted_jobs']} job(s), {failed_count} failed")

## 5) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

try:
    # Prepare output data
    output_data = {
        'batch_id': export_batch_id,
        'batch_type': batch['batch_type'],
        'batch_status': submit_result['batch_status'],
        'submitted_jobs': submit_result['submitted_jobs'],
        'failed_jobs': failed_count,
        'rdm_name': rdm_name,
        'server_name': server_name,
        'total_items_exported': len(analysis_names)
    }
    
    # Complete the step
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("EXPORT TO RDM SUBMITTED SUCCESSFULLY")
    ux.success("="*60)
    ux.info(f"\nSubmitted {submit_result['submitted_jobs']} job(s) to Moody's API")
    ux.info(f"Target RDM: {rdm_name}")
    ux.info(f"Items to export: {len(analysis_names)} ({analysis_count} analyses + {group_count} groups)")
    if failed_count > 0:
        ux.warning(f"{failed_count} job(s) failed to submit")
    ux.info(f"Batch status: {submit_result['batch_status']}")
    ux.info("\nMonitor job progress and verify export completion in Moody's.")

except Exception as e:
    ux.error(f"Step completion failed: {str(e)}")
    step.fail(str(e))
    raise